In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result
    
#RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore',category=FutureWarning)

#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.3):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\anjan\\Documents\\Emotion Exctraction From audio\\Dataset\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.2)

print((x_train.shape[0], x_test.shape[0]))

print(f'Features extracted: {x_train.shape[1]}')



#mlp
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(200,), learning_rate='adaptive', max_iter=500)
#Train the model
model.fit(x_train,y_train)
#Predict for the test set
y_pred=model.predict(x_test)
print(y_pred)
#accuracy
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)


print("Accuracy: {:.2f}%".format(accuracy*100))


(614, 154)
Features extracted: 180
['happy' 'calm' 'disgust' 'happy' 'disgust' 'calm' 'disgust' 'disgust'
 'disgust' 'happy' 'happy' 'fearful' 'fearful' 'happy' 'disgust' 'happy'
 'calm' 'disgust' 'disgust' 'disgust' 'disgust' 'disgust' 'disgust' 'calm'
 'fearful' 'happy' 'fearful' 'happy' 'fearful' 'fearful' 'happy' 'disgust'
 'happy' 'fearful' 'disgust' 'disgust' 'calm' 'fearful' 'calm' 'disgust'
 'happy' 'disgust' 'calm' 'calm' 'disgust' 'disgust' 'disgust' 'disgust'
 'calm' 'happy' 'fearful' 'fearful' 'fearful' 'happy' 'disgust' 'fearful'
 'disgust' 'happy' 'calm' 'calm' 'disgust' 'calm' 'disgust' 'calm'
 'disgust' 'calm' 'calm' 'disgust' 'disgust' 'fearful' 'fearful' 'fearful'
 'fearful' 'fearful' 'fearful' 'disgust' 'fearful' 'happy' 'calm'
 'fearful' 'disgust' 'calm' 'fearful' 'calm' 'disgust' 'happy' 'disgust'
 'fearful' 'happy' 'disgust' 'disgust' 'fearful' 'disgust' 'happy'
 'disgust' 'disgust' 'calm' 'calm' 'calm' 'disgust' 'fearful' 'disgust'
 'disgust' 'disgust' 'disgust' 